In [1]:
import pandas as pd
import numpy as np

In [5]:
import os

In [6]:
filename = "Diabetes.csv"
try:
    os.chdir(os.path.join(os.getcwd(), 'resources/PreparingDatasets'))
    print(os.getcwd())
except:
    pass

/Users/matthewbeyer/Documents (non-sync)/coursework/python-data-analysis-course/resources/PreparingDatasets


In [7]:
df = pd.read_csv(filename)

In [10]:
df.info() # this is good to run to see datatypes in each field

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [11]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
# note that in this dataset, missing data is noted as "0"

In [13]:
# if this dataset had missing data as null, you can use df.fillna(0) which will fill nulls with 0

In [14]:
# or df.dropna() which you can drop based on some parameters. look up the documentation

In [15]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]] # this creates a new dataframe based on those columns

In [16]:
df2.head()

,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


In [17]:
df2.describe()

,Glucose,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000
mean,120.894531,31.992578,33.240885,0.348958
std,31.972618,7.884160,11.760232,0.476951
min,0.000000,0.000000,21.000000,0.000000
25%,99.000000,27.300000,24.000000,0.000000
50%,117.000000,32.000000,29.000000,0.000000
75%,140.250000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


In [18]:
# describe() gives summary statistics... you should use this to validate dataset

In [19]:
# in this case, Glucose and BMI have 0 which is nonsense, so need to clean it up

In [20]:
df2.columns

Index(['Glucose', 'BMI', 'Age', 'Outcome'], dtype='object')

In [22]:
df2.columns[:-1] # all but last column

Index(['Glucose', 'BMI', 'Age'], dtype='object')

In [24]:
# so that gives the columns we want to check for 0s... since 0s in the last column (outcome) are fine

In [25]:
df2[df2.columns[:-1]] == 0 # so this will give a boolean mask of which cells in those columns are 0

,Glucose,BMI,Age
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
763,False,False,False
764,False,False,False
765,False,False,False
766,False,False,False


In [26]:
(df2[df2.columns[:-1]] == 0).any(axis=1) # "any" checks if any of the columns are 0, axis=1 is the horizontal axis

0      False
1      False
2      False
3      False
4      False
       ...  
763    False
764    False
765    False
766    False
767    False
Length: 768, dtype: bool

In [27]:
df3 = df2.loc[~(df2[df2.columns[:-1]] == 0).any(axis=1)] # the tilde inverts the mask, since we want to keep the "false"

In [28]:
df3.describe()

,Glucose,BMI,Age,Outcome
count,752.000000,752.000000,752.000000,752.000000
mean,121.941489,32.454654,33.312500,0.351064
std,30.601198,6.928926,11.709395,0.477621
min,44.000000,18.200000,21.000000,0.000000
25%,99.750000,27.500000,24.000000,0.000000
50%,117.000000,32.300000,29.000000,0.000000
75%,141.000000,36.600000,41.000000,1.000000
max,199.000000,67.100000,81.000000,1.000000


In [29]:
# now the data is clean

In [30]:
df3.groupby("Outcome") # you can put in multiple groupby attributes but this case only want to look at one

In [31]:
df3.groupby("Outcome").mean()

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.876434,31.309426
1,142.488636,35.371970,37.015152


In [34]:
# can use agg method to show multiple statistics... see for example below (although summing ages doesn't really make sense...)
df3.groupby("Outcome").agg({
    "Glucose": "mean",
    "BMI": "median", 
    "Age": "sum"
    })

,Glucose,BMI,Age
Outcome,,,
0,110.825820,30.10,15279
1,142.488636,34.25,9772


In [35]:
# can use agg method to show multiple statistics... example for just an array as the parameter...
df3.groupby("Outcome").agg(["mean", "median"])

Glucose               BMI               Age       
               mean median       mean median       mean median
Outcome                                                       
0        110.825820  107.5  30.876434  30.10  31.309426     27
1        142.488636  140.5  35.371970  34.25  37.015152     36

In [36]:
# splitting dataframe
positive = df3.loc[df3["Outcome"] == 1]
negative = df3.loc[df3["Outcome"] == 0]

In [38]:
print(positive.shape, negative.shape)

(264, 4) (488, 4)


In [39]:
df3.to_csv("clean_diabetes.csv", index=False) # by default it saves the index so this specifies not to 